自然語言處理 — 使用 N-gram 實現輸入文字預測
https://medium.com/%E6%89%8B%E5%AF%AB%E7%AD%86%E8%A8%98/%E8%87%AA%E7%84%B6%E8%AA%9E%E8%A8%80%E8%99%95%E7%90%86-%E4%BD%BF%E7%94%A8-n-gram-%E5%AF%A6%E7%8F%BE%E8%BC%B8%E5%85%A5%E6%96%87%E5%AD%97%E9%A0%90%E6%B8%AC-10ac622aab7a

In [1]:
from collections import Counter, namedtuple
import json
import re

In [2]:
DATASET_DIR = './WebNews.json'
with open(DATASET_DIR, encoding = 'utf8') as f:
    dataset = json.load(f)

In [3]:
seg_list = list(map(lambda d: d['detailcontent'], dataset))
rule = re.compile(r"[^\u4e00-\u9fa5]")
seg_list = [rule.sub('', seg) for seg in seg_list]
seg_list[1]

'桃市歲以上長者接種率六都第一明起第二輪接種請市民預先填妥評估及意願書以節省時間桃園市長鄭文燦今日下午召開疫情說明記者會時表示今日是桃園社區接種站服務歲以上原住民族歲以上長者疫苗接種的最後一日今日歲長者疫苗施打造冊人數萬人實際接種萬人接種率另有補打位不在籍位代表避免移動就近接種是正確且貼心的政策鄭市長提及桃園市疫苗接種計畫目前規劃個社區接種站及個大型接種站採分年齡分組別分流的接種模式未來將增加個大型接種站持續落實精準有序友善大接種原則市府有信心月底前能達到蔡總統所要求的覆蓋率目標鄭市長指出截至目前統計桃園歲以上長者萬人實際接種萬人接種率六都第一歲以上長者人口數萬人實際接種萬人接種率也是六都第一而全市人口數萬人已有萬人完成疫苗施打接種率為另外服務萬人不在籍長者接種疫苗鄭市長提及桃園孕婦疫苗施打專案市府於月日月日於桃園巨蛋體育館接種站服務準媽媽兩日預約總人數人實際接種人接種率達若再加上家醫院及婦產科開設的孕婦專用門診時段估計已有超過位準媽媽完成疫苗接種相信對保護準媽媽健康及母子平安有很大助益此外桃園歲以上獨居老人本市造冊人透過社工員一對一服務已有人完成施打接種率另復興區則以個衛生所及個部落接種站進行疫苗接種並配合泰雅族彩虹計程車接送服務接種率將近成鄭市長談到市府也提供多元的貼心接種服務感謝大車隊出動輛防疫計程車協助接種載送服務防疫車隊執行趟次疫苗接種愛心計程車萬趟次孕婦接送任務趟次另提供輛復康巴士接送身障或行動不便的長者至接種站共計服務趟次鄭市長表示疫苗意願登記及預約施打計畫未來第二輪至第四輪皆提供疫苗其中第二輪施打時間自明日至月日早上時開始於個社區接種站及個桃園巨蛋體育館大型接種站進行接種服務截至今日中午已有萬人預約登記市府將於今日下午時前完成最後統計鄭市長指出明日服務對象除了第類高風險疾病者罕見疾病及重大傷病者第類歲以上成人等公費對象外有預約的歲以上長者也可以來接種相信全民接種計畫的預約系統上線後施打年齡層可以逐步往下此外市府也鼓勵民眾提早下載或至里辦索取疫苗接種評估及意願書網址在家先行填寫以減少現場等候時間鄭市長提及第三輪疫苗意願施打時間為月日至月日市府將維持個社區接種站每站每日可服務至少人同時將擴增大型接種站服務除了原有的桃園巨蛋體育館外將新增陸軍專科學校及體育大學林口體育館站個大型接種站預計日內可服務萬位民眾加上家醫院家診所家衛生所等合約醫療院所加入服務預估第

In [ ]:
'''
N-gram 實作
首先將每一篇文章做 tokenize；
而我們想知道一篇文章的開頭與結尾，所以在 tokenize 之後，在前後分別加上 <s> 和 </s> 作為標示。
而在計算每個字出現的機率後，可以利用 set，將重複的字與機率去除
'''

In [4]:
def ngram(documents, N=2):
    ngram_prediction = dict()
    total_grams = list()
    words = list()
    Word = namedtuple('Word', ['word', 'prob'])

    for doc in documents:
        split_words = ['<s>'] + list(doc) + ['</s>']
        # 計算分子
        [total_grams.append(tuple(split_words[i:i+N])) for i in range(len(split_words)-N+1)]
        # 計算分母
        [words.append(tuple(split_words[i:i+N-1])) for i in range(len(split_words)-N+2)]
        
    total_word_counter = Counter(total_grams)
    word_counter = Counter(words)
    
    for key in total_word_counter:
        word = ''.join(key[:N-1])
        if word not in ngram_prediction:
            ngram_prediction.update({word: set()})
            
        next_word_prob = total_word_counter[key]/word_counter[key[:N-1]]
        w = Word(key[-1], '{:.3g}'.format(next_word_prob))
        ngram_prediction[word].add(w)
        
    return ngram_prediction

In [ ]:
'''
訓練模型與排序
我們使用的模型是 trigram，也就是計算接在兩個字之後第三個字的機率。接著，可以對結果進行排序，
因此在預測下一個字時，能夠直接取得前幾個最高機率的字。
'''

In [5]:
tri_prediction = ngram(seg_list, N=3)
for word, ng in tri_prediction.items():
    tri_prediction[word] = sorted(ng, key=lambda x: x.prob, reverse=True)

In [ ]:
'''
預測輸入的下一個字
最後，我們使用模型來預測接下來可能會出現的文字
'''

In [7]:
text = '未來'
next_words = list(tri_prediction[text])[:5]
for next_word in next_words:
    print('next word: {}, probability: {}'.format(next_word.word, next_word.prob))

next word: 市, probability: 0.0954
next word: 將, probability: 0.0882
next word: 也, probability: 0.0713
next word: 可, probability: 0.0348
next word: 桃, probability: 0.0312
